# Natalia Organek - laboratorium 6 - Language Modelling

In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

## Download three Polish models from the Huggingface repository.

Użyte modele:
 - dkleczek/bert-base-polish-cased-v1
 - Geotrend/distilbert-base-pl-cased
 - allegro/herbert-base-cased

In [4]:
class Model:
    
    def __init__(self, name):
        self.name = name
        self.tokenizer = AutoTokenizer.from_pretrained(name),
        self.model = AutoModelForMaskedLM.from_pretrained(name)

In [5]:
model_names = [
    'dkleczek/bert-base-polish-cased-v1', 
    'Geotrend/distilbert-base-pl-cased',
    'allegro/herbert-base-cased'
]

In [6]:
models = [Model(name) for name in model_names]

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a B

## Produce the predictions for the following sentences (use each model and check 5 predictions):
 - (M) Warszawa to największe [MASK].
 - (D) Te zabawki należą do [MASK].
 - (C) Policjant przygląda się [MASK].
 - (B) Na środku skrzyżowania widać [MASK].
 - (N) Właściciel samochodu widział złodzieja z [MASK].
 - (Ms) Prezydent z premierem rozmawiali wczoraj o [MASK].
 - (W) Witaj drogi [MASK].

In [34]:
sentences = [
    'Warszawa to największe [MASK].',
    'Te zabawki należą do [MASK].',
    'Policjant przygląda się [MASK].',
    'Na środku skrzyżowania widać [MASK].',
    'Właściciel samochodu widział złodzieja z [MASK].',
    'Prezydent z premierem rozmawiali wczoraj o [MASK].',
    'Witaj drogi [MASK].'
]

In [30]:
def prediction(model, sentence, k=5):
    sentence = sentence.replace('[MASK]', model.tokenizer[0].mask_token)
    
    inputs = model.tokenizer[0](sentence, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == model.tokenizer[0].mask_token_id)[1]
    
    token_logits = model.model(**inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]
    
    top_k_tokens = torch.topk(mask_token_logits, k, dim=1).indices[0].tolist()
    
    return [model.tokenizer[0].decode([token]) for token in top_k_tokens]

In [31]:
def prettify(sentences, models):
    for sentence in sentences:
        results = {}
        for model in models:
            results[model.name] = prediction(model, sentence)
        df = pd.DataFrame(results)
        print(sentence)
        display(df)
        print()

In [99]:
prettify(sentences, models)

Warszawa to największe [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,miasto,miasto,miasto
1,województwo,miasta,lotnisko
2,lotnisko,Miasto,centrum
3,miasteczko,centrum,miasta
4,państwo,dzielnicy,atrakcje



Te zabawki należą do [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,ciebie,klasyfikacji,rodziny
1,mnie,gry,nas
2,nas,rodziny,nich
3,pana,grupy,najlepszych
4,niego,zespołu,.



Policjant przygląda się [MASK].


...

,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,bronią,Warszawy,samochodu
1,tyłu,pochodzenia,włamaniem
2,ulicy,Niemiec,auta
3,bliska,Łodzi,kierowcą
4,zewnątrz,Rosji,parkingu



Prezydent z premierem rozmawiali wczoraj o [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,tym,prezydenta,przyszłości
1,Polsce,referendum,Polsce
2,budżecie,władze,bezpieczeństwie
3,ASF,premiera,polityce
4,ustawie,LGBT,Warszawie



Witaj drogi [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,chłopcze,drogi,Łukasz
1,przyjacielu,narodowej,Boże
2,bracie,polskiej,człowieku
3,kolego,##wej,Karol
4,synu,wschodniej,Marcin


## Check the model predictions for the following sentences (using each model):
- Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].
- Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].


In [100]:
sentences = [
    'Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].',
    'Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].'
]

In [101]:
prettify(sentences, models)

Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,zgodził,było,poddał
1,bał,stanie,zdziwił
2,dowiedział,udało,dowiedział
3,pojawił,tylko,zastanawiał
4,zabił,odbył,przyznał



Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,zgodziła,było,dowiedziała
1,bała,udało,przyznała
2,dowiedziała,stanie,bała
3,pojawiła,tylko,zmieniła
4,zabiła,ma,zgodziła


## Check the model predictions for the following sentences:
- [MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
- W wakacje odwiedziłem [MASK], który jest stolicą Islandii.
- Informatyka na [MASK] należy do najlepszych kierunków w Polsce.

In [102]:
sentences = [
    '[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.',
    'W wakacje odwiedziłem [MASK], który jest stolicą Islandii.',
    'Informatyka na [MASK] należy do najlepszych kierunków w Polsce.'
]

In [103]:
prettify(sentences, models)

[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,Woda,Na,Woda
1,Mięso,We,Słońce
2,Słońce,Od,Ziemia
3,Nie,Maja,Następnie
4,Ziemia,Zmarł,Ciało



W wakacje odwiedziłem [MASK], który jest stolicą Islandii.


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,kraj,kraju,Kraków
1,Cypr,pochodzi,Oslo
2,Meksyk,flag,Londyn
3,Gibraltar,wody,Gdańsk
4,Wellington,referendum,Toruń



Informatyka na [MASK] należy do najlepszych kierunków w Polsce.


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,wsi,stacji,pewno
1,świecie,Uniwersytecie,AGH
2,żywo,Ziemi,UW
3,pewno,terenie,studiach
4,odległość,ulicy,UMK


### Which of the models produced the best results?
Zacznę od tego, że najgorsze wyniki dał model Geotrend - dopasowania często były niegramatyczne i/lub pozbawione sensu. Między modelami od allegro i dkleczek ciężko jest wybrać - model dkleczek lepiej radzi sobie z gramatyką (praktycznie każde dopasowanie było poprawne pod tym względem), ale odpowiedzi od allegro bywają często lepsze w odniesieniu do odzwierciedlenia świata - przykład zdania ze skrzyżowaniem:

Na środku skrzyżowania widać [rzekę, ulicę, drzewo, wodę, las]. - dkleczek
Na środku skrzyżowania widać [rondo, samochody, radiowóz, samochód, wiadukt]. - allegro.

Często też odpowiedzi od allegro są mniej ogólne, w dodatku ten model lepiej poradził sobie w zadaniu 3 - teście z wiedzy nt świata. Myślę więc, że wybór między tymi modelami zależy od tego, do czego chcemy je wykorzystać - jeśli zależy nam na poprawności gramatycznej - dkleczek będzie lepszym wyborem, jeśli jednak na ciekawszych, dokładniejszych odpowiedziach (bo np. gramatykę możemy poprawić sami) to allegro będzie lepszym wyborem.



### Was any of the models able to capture Polish grammar?

Tak, był to model **dkleczek** - bezproblemowo dopasował wyrazy w odpowiedniej formie gramatycznej
zad 1 - rzeczowniki:
mimo że nie zawsze był to przypadek, którego oczekiwaliśmy (Właściciel samochodu widział złodzieja z **ulicy** - gdy oczekiwany był narzędnik, ale nie precyzowałam tego nigdzie w zapytaniu do modelu, więc tak naprawdę miał on dowolność dopasowania), ale dalej były to w pełni poprawne gramatycznie dopasowania. 
zad 2 - czasownik:
Również poradził sobie bardzo dobrze, rozpoznał, jaki powinien być rodzaj czasownika.

Model od **allegro** też poradził sobie dobrze, przy rzeczownikach największy problem miał przy wołaczu, gdy do zdania 'Witaj drogi ...' dopasowywał częściej mianownik - tak jakby był wyuczony na bardziej potocznej niż poprawnej polszczyźnie. Miał też pojedyncze potknięcia przy mianowniku (Warszawa to największe **miast**.) i dopełniaczu (Te zabawki należą do **.** .) Jeśli chodzi o czasownik - poradził sobie równie dobrze jak model dkleczek.

Najgorzej poradził sobie pod względem gramatycznym (i nie tylko) model z **Geotrend**. Wygląda na to, że dla niego są przypadki łatwiejsze, z którymi radzi sobie całkiem nieźle (dopełniacz, narzędnik), ale niektóre wypadły dość słabo (mianownik, biernik, miejscownik). Jeśli chodzi o czasowniki z zadania 2 - w kontekście tych zdań nie dopasował poprawnie ani jednego wyrazu.

### Was any of the models able to capture long-distant relationships between the words?
Tak, patrząc na przykład z zadania 2 - dobrze poradziły sobie modele dkleczek i allegro. Dopełnienia są nie tylko poprawne gramatycznie, ale i w większości (a na upartego wszystkie) mogłyby być użyte jako poprawne zdania (Nawet "Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie **zabił/dowiedział**."). Spośród tych dwóch ciężko wybrać zwycięzcę, ale skłaniam się ku allegro, ponieważ poza "dowiedział", pozostałe wybory były nadzwyczaj trafne. Jeśli chodzi o model Geotrend - poradził sobie z tym zadaniem bardzo słabo.

Z przykładów "Właściciel samochodu widział złodzieja z [MASK]." i "Prezydent z premierem rozmawiali wczoraj o [MASK]." można wywnioskować że model Geotrend działa lepiej (od samego siebie, nie od pozostałych modeli) dla bliższych słów 

W pierwszym zdaniu opisywał skąd był złodziej (Warszawy, Niemiec - czyli łączył dopasowania jedynie z poprzedzającym wyrazem **z**), podczas gdy pozostałe modele opisywały skąd złodziej był widziany (samochodu/ulicy/tyłu), albo co/kogo przy sobie miał (bronią/kierowcą) tak jakby brały pod uwagę odleglejsze słowo **widział** albo **złodzieja z**. 

W drugim z tych zdań Geotrend dopasowuje niegramatyczne zdania, ale jeśli weźmiemy pod uwagę, że dopasowuje tylko do poprzedzającego wyrazu **o** to wybór dopasowań zaczyna mieć sens ([pytać] **o** prezydenta, referendum, władze, premiera, LGBT) - w kontekście prezydenta i premiera dopasowania są dobre, zabrakło tylko wzięcia pod uwagę czasownika "rozmawiali" i dopasowania do niego odpowiedniej formy - z czym problemu nie miały pozostałe modele.

### Was any of the models able to capture world knowledge?
Żaden z modeli nie dopasował dobrze wszystkich zapytań o wiedzę (zad. 3). Najlepiej tym razem wypadł model od allegro - dobrze dopasował, że to woda ma opisane właściwości (była to pierwsza odpowiedź, przewinęło się też słońce i ciało) i dość dokładnie dopasował, jaka informatyka jest najlepsza w Polsce (oprócz ogólnikowych odpowiedzi "na studiach" i "na pewno", podał również **przykładowe** uczelnie - AGH/UW/UMK). Na pytanie o stolicę Islandii nie udzieliło poprawnej odpowiedzi ALE dopasowało same miasta, co nie udało się pozostałym modelom.

Model dkleczek nie był już tak dobry - dobrze wybrał wodę jako dopasowanie do pierwszego zdania, ale w zdaniu ze stolicą Islandii przewinął się wyraz "kraj" oraz "Cypr", co jest bardziej niepoprawne od odpowiedzi modelu allegro, gdzie pojawiły się same miasta. Jeśli chodzi o zdanie z najlepszą informatyką - nie dał żadnej konkretnej odpowiedzi, wszystkie były poprawne gramatycznie, semantycznie w większości też, ale z wiedzą o świecie niewiele to miało wspólnego (Informatyka na wsi należy do najlepszych kierunków w Polsce. - ogólnikowe, ciężko doszukać się też uniwersytetów z informatyką na wsi).


Wyniki modelu Geotrend ciężko jest komentować - są niegramatyczne i nielogiczne - najlepszym przykładem są odpowiedzi do pierwszego zdania 

[Na, We, Od, Maja, Zmarł] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.

W odpowiedziach o stolicę Islandii też ciężko doszukać się sensu.

Pewien przebłysk jest jednak w odpowiedzi o najlepszą Informatykę - **Uniwersytecie** zaczyna się wielką literą - być może jest to pierwsza część nazwy uniwersytetu.


 Z ciekawości sprawdziłam, czy może zdania zbudowane pod casual match lepiej zadziałają w zdaniach z zadania trzeciego (pod wnioskami), ale wyniki są nawet ciut gorsze (w dodatku trochę przykro jest się dowiedzieć, że dla allegro nawet nasza Informatyka jest na sprzedaż ;) ).



### What are the most striking errors made by the models?

 - błędy gramatyczne - złe przypadki (np. "Warszawa to największe miast." w wykonaniu allegro, "Prezydent z premierem rozmawiali wczoraj o prezydenta" w wykonaniu Geotrend)
 - Znak interpunkcyjny przed . w zdaniu (Te zabawki należą do . . - allegro)
 - Wyrazy typu ##cy (maski?) jako odpowiedź od Geotrend
 - Brak poprawności semantycznej (Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie tylko. Na środku skrzyżowania widać miejsca. W wakacje odwiedziłem referendum, który jest stolicą Islandii. - Geotrend)
 - Brak poprawności w sensie wiedzy o świecie - niedopasowanie stolicy Islandii.
 
 

In [40]:
sentences = [
    'W przyszłości zostanę [MASK].',
    'Substancja, która wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza to [MASK].',
    'W wakacje odwiedziłam stolicę Islandii - [MASK].',
    'Najlepsza informatyka w Polsce jest na [MASK].'
]
prettify(sentences, models)

W przyszłości zostanę [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,ojcem,dzieci,trenerem
1,lekarzem,muzeum,lekarzem
2,prezydentem,szkoły,dyrektorem
3,dziadkiem,miejsca,nauczycielem
4,pisarzem,nowy,redaktorem



Substancja, która wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza to [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,ok,stała,woda
1,azot,##czna,.
2,tlen,jest,tlen
3,woda,##rki,węgiel
4,1,##cznych,lód



W wakacje odwiedziłam stolicę Islandii - [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,Oslo,Samoa,Oslo
1,Helsinki,Finlandia,Londyn
2,Ateny,Island,Kraków
3,Singapur,Anglii,Warszawę
4,Budapeszt,Niemcy,Gdańsk



Najlepsza informatyka w Polsce jest na [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/distilbert-base-pl-cased,allegro/herbert-base-cased
0,świecie,Ziemi,sprzedaż
1,rynku,internet,rynku
2,ukończeniu,igrzyskach,ukończeniu
3,miejscu,Uniwersytecie,świecie
4,liście,świecie,szczycie
